In [ ]:
#语言模型
#为了计算语言模型，我们需要计算词的概率，以及一个词在给定前几个词的情况下的条件概率，即语言模型参数。
#P(w1,w2,w3,w4)=P(w1)P(w2∣w1)P(w3∣w1,w2)P(w4∣w1,w2,w3)
#n元语法,词的出现之和前面n个词有关
#时间步t的词wt基于前面所有词的条件概率只考虑了最近时间步的n−1个词
#词的概率可以是词在训练数据集中的相对词频（词出现的次数/训练集总数）
#P(w2|w1):w2,w1两词相邻的频率/w1词频，p(w2|w1) = p(w2,w1)/p(w1)

#循环神经网络
#和不含隐藏层的相比 
#Ht=ϕ(X_tW_xh+H_{t−1}W_hh+b_h)
#增加了一个变量上一时间步的隐藏变量Ht−1
#隐藏状态中XtWxh+Ht−1Whh的计算等价于Xt与Ht−1连结后的矩阵乘以Wxh与Whh连结后的矩阵。

In [1]:
import torch
X,W_xh = torch.randn(3,1),torch.randn(1,4)
H,W_hh = torch.randn(3,4),torch.randn(4,4)
torch.matmul(X,W_xh)+torch.matmul(H,W_hh)


tensor([[-0.9708, -1.3721, -0.1813, -1.2993],
        [-0.6693,  1.3225, -2.5003,  2.9734],
        [ 1.1489, -1.4049,  1.4557,  1.4024]])

In [2]:
torch.matmul(torch.cat((X,H),dim=1),torch.cat((W_xh,W_hh),dim=0))

tensor([[-0.9708, -1.3721, -0.1813, -1.2993],
        [-0.6693,  1.3225, -2.5003,  2.9734],
        [ 1.1489, -1.4049,  1.4557,  1.4024]])

In [6]:
#语言模型
import torch
import random
import zipfile

with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
        
corpus_chars[:40]


'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [7]:
#把换行符替换成空格，然后仅使用前1万个字符来训练模型。
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:10000]

In [53]:
#建造字符索引
#每个字符映射成一个从0开始的连续整数，又称索引，来方便之后的数据处理。
#不同字符取出构造词典
idx_to_char = list(set(corpus_chars)) #给不同的字符编号
print(idx_to_char[:20])
char_to_idx = dict([(char,i) for i,char in enumerate(idx_to_char)]) #建造字典
vocab_size = len(char_to_idx)
#训练集中每个字符转化为索引
corpus_indices = [char_to_idx[char] for char in corpus_chars]#根据字典查每个字对应的索引
sample = cropus_indices[:20]
print('chars:',''.join([idx_to_char[idx] for idx in sample]))
print('indices:',sample)

['宣', '否', '泊', '反', '抛', 'C', '田', '准', '奇', '确', '毫', '溪', ' ', '狂', '铅', '回', '诗', '告', '盲', '袋']
chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [936, 918, 269, 126, 281, 854, 12, 936, 918, 646, 569, 416, 749, 167, 339, 730, 12, 936, 918, 646]


In [16]:
#时序数据采样
#随机
#批量大小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。
#每个样本是原始序列上任意截取的一段序列。
#相邻的两个随机小批量在原始序列上的位置不一定相毗邻。因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。
def data_iter_random(cropus_indices,batch_size,num_steps,device=None):
    num_examples = (len(corpus_indices)-1)//num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    #返回从pos开始长为num_steps的序列
    def _data(pos):
        return cropus_indices[pos:pos+num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for i in range(epoch_size):
        #每次读取batch_size个随机样本
        i = i*batch_size
        batch_indices = example_indices[i:i+batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps+1) for j in batch_indices]
        yield torch.tensor(X,dtype=torch.float32,device=device),torch.tensor(Y,dtype=torch.float32,device=device)

my_seq = list(range(30))
for X,Y in data_iter_random(my_seq,batch_size=2,num_steps=6):
    print('X:',X,'\nY:',Y,'\n')
    
    
    

X: tensor([[18., 19., 20., 21., 22., 23.],
        [12., 13., 14., 15., 16., 17.]], device='cuda:0') 
Y: tensor([[19., 20., 21., 22., 23., 24.],
        [13., 14., 15., 16., 17., 18.]], device='cuda:0') 

X: tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [ 6.,  7.,  8.,  9., 10., 11.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [ 7.,  8.,  9., 10., 11., 12.]], device='cuda:0') 



In [25]:
#相邻采样
#可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices,dtype=torch.float32,device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:,i:i+num_steps]
        Y = indices[:,i+1:i+num_steps+1]
        yield X,Y

In [26]:
for X,Y in data_iter_consecutive(my_seq,batch_size=2,num_steps=6):
    print('X:',X,'\nY:',Y,'\n')


X: tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]], device='cuda:0') 

X: tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]], device='cuda:0') 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]], device='cuda:0') 



In [28]:
#循环神经网络
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2l.torch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [29]:
#如果一个字符的索引是整数i, 那么我们创建一个全0的长为N的向量，并将其位置为i的元素设成1。该向量就是对原字符的one-hot向量。
def one_hot(x,n_class,dtype=torch.float32):
    ## X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res
x = torch.tensor([0,2])#对应的词典里面的index
one_hot(x,vocab_size)#转化成词向量（长度为词典长度的向量对应位置为1）


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [35]:
#采样的小批量形状是（批量大小，时间步数）
#函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵
def to_onehot(X,n_class):
    # X shape:(batch,seq_len),output:seq_len elements of (batch,n_class)
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]
X = torch.arange(10).view(2,5)#（批量大小，时间步数）
print(X)
inputs = to_onehot(X,vocab_size)#把每个批量里的样本转化成词向量
print(len(inputs),inputs[0].shape)


tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])
[tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
5 torch.Size([2, 1027])


In [31]:
num_inputs,num_hiddens,num_outputs = vocab_size,256,vocab_size
print('will use',device)
#初始化模型参数
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    # 隐藏层参数
    W_xh = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens,device=device,requires_grad=True))
    #输出层参数
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,device=device,requires_grad=True))
    return nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])

will use cuda


In [33]:
#定义模型
#返回初始化的隐藏状态。它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组。、
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)
def rnn(inputs,state,params):
    W_xh,W_hh,b_h,W_hq,b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X,W_xh)+torch.matmul(H,W_hh)+b_h)
        Y = torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)
state = init_rnn_state(X.shape[0],num_hiddens,device)
inputs = to_onehot(X.to(device),vocab_size)
params = get_params()
outputs,state_new = rnn(inputs,state,params)
print(len(outputs),outputs[0].shape,state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


In [62]:
#定义预测函数
#以下函数基于前缀prefix（含有数个字符的字符串）来预测接下来的num_chars个字符。
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,
                 num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state = init_rnn_state(1,num_hiddens,device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        #将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        #计算输出和更新隐藏状态
        (Y,state) = rnn(X,state,params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

predict_rcnn('在一起',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx)

'在一起断友陷温丛但梭优耿选'

In [60]:
#梯度裁剪
#为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。
#假设我们把所有模型参数梯度的元素拼接成一个向量 g，并设裁剪的阈值是θ。裁剪后的梯度
#min(θ/∥g∥,1)g
def grad_clipping(params,theta,device):
    norm = torch.tensor([0.0],device=device)
    for param in params:
        norm += (param.grad.data**2).sum() #算∥g∥
    norm = norm.sqrt().item()
    if norm > theta:#θ/∥g∥<1
        for param in params:
            param.grad.data *= (theta/norm)

In [58]:
#困惑度，使用困惑度（perplexity）来评价语言模型的好坏。
#    最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
#     最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
#     基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。
#定义模型训练函数
#    使用困惑度评价模型。
#     在迭代模型参数前裁剪梯度。
#     对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关讨论可参考6.3节（语言模型数据集（周杰伦专辑歌词））。
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps)#corpus_indices每个字对应的索引
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  
            # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()

            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)#批量大小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())

            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [49]:
num_epochs,num_steps,batch_size,lr,clipping_theta = 250,35,32,1e2,1e-2
pred_period,pred_len,prefixes = 50,50,['分开','不分开']

In [63]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, True, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes)

epoch 50, perplexity 67.997527, time 0.10 sec
 - 分开 我想要 你爱我 我想就 一直两 三颗四 一颗四 我想就的让我有动 什么的让我疯狂的可爱女人 坏坏的
 - 不分开 快颗我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别子我 别
epoch 100, perplexity 9.724555, time 0.10 sec
 - 分开 一颗用美三 我有往事些够 如果我遇见你是一场悲剧 我可以好生命你的 不要再 爱你的睛不著我 爱你的
 - 不分开粥 我不要再想 我不能再想 我不 我不 我不能再想你 不知不觉 你已经离开我 不知不觉 我已了这节奏
epoch 150, perplexity 2.806186, time 0.10 sec
 - 分开 一颗两步三步四步望习天 看星星 一颗两颗三颗四颗 连成线背著背默默许下心愿 看远方的星是否听的见 
 - 不分开吗 我后你爸多 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要
epoch 200, perplexity 1.623627, time 0.10 sec
 - 分开 一只在它心 谁让它停 在它怕动屋 白慢蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋
 - 不分开期 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要
epoch 250, perplexity 1.351219, time 0.10 sec
 - 分开 一只在停留 谁让它停留的 为什么我女朋友场外加油 你却还让我出糗 从小睡耳濡目染么找也找不着 心血
 - 不分开扫 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱


In [1]:
#rnn简介实现
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F

import sys
# sys.path.append("..")
import d2lzh_pytorch as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
(corpus_indices,char_to_idx,idx_to_char,vocab_size) = d2l.load_data_jay_lyrics()



In [3]:
#定义模型
#构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层rnn_layer
num_hiddens = 256
rnn_layer = nn.RNN(input_size=vocab_size,hidden_size = num_hiddens) #rnn_layer的输入形状为(时间步数, 批量大小, 输入个数)
#其中输入个数即one-hot向量长度（词典大小）
num_steps = 35
batch_size = 2
state = None
print(vocab_size)
X = torch.rand(num_steps,batch_size,vocab_size)#(35,2,1027)
Y,state_new = rnn_layer(X,state)
print(Y.shape,len(state_new),state_new[0].shape)
#nn.RNN实例在前向计算返回的隐藏状态指的是隐藏层在最后时间步的隐藏状态：当隐藏层有多层时，每一层的隐藏状态都会记录在该变量中；

1027
torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [4]:
rnn_layer2 = nn.LSTM(input_size=vocab_size,hidden_size = num_hiddens) #rnn_layer的输入形状为(时间步数, 批量大小, 输入个数)
#其中输入个数即one-hot向量长度（词典大小）
X2 = torch.rand(num_steps,batch_size,vocab_size)#(35,2,1027)
Y2,state_new2 = rnn_layer2(X,state)
print(Y2.shape,len(state_new2),state_new2[0].shape)
#对于像长短期记忆（LSTM），隐藏状态是一个元组(h, c)，即hidden state和cell state。

torch.Size([35, 2, 256]) 2 torch.Size([1, 2, 256])


In [9]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super(RNNModel,self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size,vocab_size)
        self.state = None
    def forward(self,inputs,state): #inputs:(batch,seq_len)
        #获取one-hot向量表示
        X = d2l.to_onehot(inputs,self.vocab_size)
        Y,self.state = self.rnn(torch.stack(X),state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1,Y.shape[-1]))
        return output,self.state
        

In [13]:
#模型训练
#预测函数定义
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,device,idx_to_char,char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars+len(prefix)-1):
        X = torch.tensor([output[-1]],device=device).view(1,1)
        if state is not None:
            if isinstance(state,tuple):#LSTM ,state:(h,c)
                state = (state[0].to(device),state[1].to(device))
            else:
                state = state.to(device)
        (Y,state) = model(X,state)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [14]:
model = RNNModel(rnn_layer,vocab_size).to(device)
predict_rnn_pytorch('分开',10,model,vocab_size,device,idx_to_char,char_to_idx)

'分开梦飞话话话话话话话话'

In [19]:
#训练函数
def train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_inidices,
                                 idx_to_char,char_to_idx,num_epochs,num_steps,lr,clipping_theta,batch_size,
                                 pred_period,pred_len,prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum,n,start = 0.0,0,time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices,batch_size,num_steps,device)
        for X,Y in data_iter:
            if state is not None:
                if isinstance(state,tuple):
                    state = (state[0].detach(),state[1].detach())
                else:
                    state = state.detach()
            (output,state) = model(X,state) #output:形状为(num_steps * batch_size,vocab_size)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y,0,1).contiguous().view(-1)
            l = loss(output,y.long())
            
            optimizer.zero_grad()
            l.backward()
            #梯度裁剪
            d2l.grad_clipping(model.parameters(),clipping_theta,device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
            
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))
            

In [20]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 9.722527, time 0.03 sec
 - 分开始我的 不要 我不要再想  不要再这样打我妈妈 想想你 我不到 一个你 我不了我想要你想你 我不要再
 - 不分开 我想要你的手 不知 不 我有了我想你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人
epoch 100, perplexity 1.271185, time 0.03 sec
 - 分开不你的模样  想要有一点秀动 猎物死了它比谁都难过 印地安斑鸠 会学人开口 仙人掌怕羞 蜥蝪横著走 
 - 不分开 我不会平原 我后再这样打我妈 你爱我 我爸 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了
epoch 150, perplexity 1.064431, time 0.03 sec
 - 分开不你的模样  想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想
 - 不分开 我有没有你 我想要这样没有你  我我妈妈常是没有你 没有 我不要再想你 爱情来的太快就像龙卷风 离
epoch 200, perplexity 1.032481, time 0.03 sec
 - 分开不你的模样  想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想
 - 不分开 我有 难过 是因为闷了很久 是因为想了太多 是心理起了作用 你说 苦笑常常陪着你 在一起有点勉强 
epoch 250, perplexity 1.020024, time 0.03 sec
 - 分开 你在操纵主义 太彻底 让我连恨都难以下笔 将真心抽离写成日记 像是一场默剧 你的完美主义 太彻底 
 - 不分开 我有没有你甘 我 一个一只心你 我不要去 手 我和你看棒球 想这样没担忧 唱着歌 一直走 我想就这


In [ ]:
#关于为什么要进行梯度裁剪
#h_t=W_hx*x_t+W_hh*ht−1,
#ot=W_qh * h_t.
#L=T/1*∑_t=1^T ℓ(o_t,y_t).
#L只通过oT依赖最终时间步T的隐藏状态hT